# EDA

This notebook contains an EDA of King County Housing Data.

We focus on the specific needs of our client, the __Buyer Larry Sanders__, which have been documented as follows: 

Waterfront, limited budget, nice & isolated but central neighborhood without kids (but got some of his own, just doesn't want his kids to play with other kids .. because of germs)


## Set up

### Import necessary libraries

In [ ]:
#import necessary libraries
#import warnings
#warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import missingno as msno

sns.set()

# from matplotlib.ticker import PercentFormatter
# plt.rcParams.update({ "figure.figsize" : (8, 5),"axes.facecolor" : "white", "axes.edgecolor":  "black"})
# plt.rcParams["figure.facecolor"]= "w"
# pd.plotting.register_matplotlib_converters()

#round all floats to 3 decimals
pd.set_option('display.float_format', lambda x: '%.3f' % x)

### Importing the Data

We were provided the data and import it directly from the data folder. However, it can also be found on [kaggle](https://www.kaggle.com/datasets/harlfoxem/housesalesprediction).
We assign the data to a DataFrame called df_housing.

In [ ]:
#loading data into DataFrame
df_housing = pd.read_csv('data/King_County_House_prices_dataset.csv')

# # as column 'condition' is truncated in the display, disable column truncation
# pd.set_option('display.max_columns', None) 
# pd.set_option('display.width', None) 
# pd.set_option('display.max_colwidth', -1)

## Examining the Data

Let's first take a look at the data provided. 

We take a look at the first 10 entries, thereby also noting the column headers and some first impression on indexing and the format of the observations.

In [ ]:
#examining the data
df_housing.head(10)

We check for duplicates among the observations that would have to be removed to avoid unequal influences of the observations in our analysis to come. We notice that there are no duplicates.

In [ ]:
# Check for duplicates - no duplicates!
df_housing.duplicated().value_counts()

In [ ]:
df_housing.id.nunique()

In [ ]:
df_housing.groupby('id').count().query('price>1')

In [ ]:
df_housing.query('id==1000102')

Let's find out how many observations and characteristics/columns we are dealing with.

In [ ]:
df_housing.shape

There are 21597 observations and 21 columns.

How are the observations indexed?

In [ ]:
# how is the data indexed?
# standard numerical indexes starting at 0 with step=1, last index = 21596
df_housing.index

The observations are indexed by integers, starting at 0 with step 1, the last entry is 21596.

What are the column names, what are the data types of the columns and do the columns contain missing values?

In [ ]:
df_housing.info()
#21597 entries, 21 columns
# there are NaN entries

We take a first look at statistical information on our data.

In [ ]:
df_housing.describe()

### First Observations/Questions

1. There is an outlier 33 in the bedrooms column.
1. There might be an outlier 8 in the bathrooms column.
1. There might be an outlier 13540 in sqft_living.
1. What is the grading system?
1. what does 'view' mean?
1. How is the condition rated?
2. Missing values in the waterfront, view, yr_renovated columns
1. sqft_basement has non-numerical entries (e.g. ?)
5. The date column is in string format and needs to be changed to datetime
8. df.describe is to be used with caution as it doesn't work with missing data (NaN)
1. column names are already in Snake Case and meaningful, no renaming necessary
1. What are half floors?
1. What are .25 bathrooms?

Researching for King County building standards and information on the data provided yields answers to some of the questions:
https://info.kingcounty.gov/assessor/esales/Glossary.aspx
https://www.kaggle.com/datasets/harlfoxem/housesalesprediction/discussion/207885

* "grade": An index from 1 to 13, where 1-3 falls short of building construction and design, 7 has an average level of construction and design, and 11-13 have a high quality level of construction and design.
* "view": Overall view quality, varying from 0 to 4. 0 = No view, 1 = Fair 2 = Average, 3 = Good, 4 = Excellent
* "condition": An index from 1 to 5 on the condition of the apartment.
* "floor" denotes the floor location on which the unit is located. This is possible on half floor levels.
* "bathrooms": A full bath contains at least one sink, one toilet, a shower and a bath. A 0.75 bathroom has either a shower or a bath. A 0.25 bathroom only contains a toilet, etc.

## Data Cleaning

### Looking at Outliers

We will now take a closer look at the outliers to decide how to treat them. 
We start with "33 bedrooms".

In [ ]:
#look at outlier bedrooms
df_housing.query("bedrooms==33")
#conclusion: does not match size of house, faulty data, line should be removed
# remove either via index df_housing.query("bedrooms==33").index or via combining with query command

Within the observation of the outlier "33 bedrooms", the number of bedrooms does not match the other characteristics of the observation, e.g. size of the house or number of bathrooms. Hence, we treat it as faulty data and decide that the line should be removed.

In [ ]:
#remove line with 33 bedrooms
df_housing.drop(df_housing.query("bedrooms==33").index, inplace=True)
# reset index inplace
df_housing.reset_index(inplace=True, drop=True)

In [ ]:
# check if successfully removed
df_housing.shape

In [ ]:
df_housing.describe()

We will now look at the observations with at least 7 bedrooms. Do they appear to be correct data?

In [ ]:
#look at outlier bedrooms
df_housing.query("bedrooms>=7").describe()
# nothing suspicious, data remains. 
# House with 7 bedrooms, 1 bathroom: fits sqft_living, data remains since possibly faulty bathroom characteristic not relevant to client

What springs to our attention is the house with 7 bedrooms, yet 1 bathroom only. We take a closer look and notice that the large amount of bedrooms fits other criteria, such as sqft_living. The data remains in our dataset since the possibly faulty bathroom characteristic is not relevant to our client. Overall, the data with at least 7 bedrooms does not seem suspicious. The same holds for "sqft_living=13540".

In [ ]:
df_housing.query("bedrooms==7 and bathrooms==1")
# bedrooms fit sqft_living, data remains

In [ ]:
#look at outlier bathrooms
df_housing.query("bathrooms==8")
#conclusion: matches large number of bedrooms and very high price, data should remain in the data set

In [ ]:
#look at outlier sqft_living
df_housing.query("sqft_living==13540")
#conclusion: matches large number of bedrooms, bathrooms and very high price, data should remain in the data set

### Data Types and Transforming Data

We can transform the data type in our date column to datetime format.

Since yr_built and yr_renovated are not of relevance to our client, we do not need to work on them. Changing the format of the latter would be more complicated because of missing values, hence, we refrain from changing the format.

In [ ]:
# change "date" dtype to datetime with format %Y/%m/%d
df_housing['date'] = pd.to_datetime(df_housing['date'], format='%m/%d/%Y')

We can now easily sort the date column and learn that the data contains the sales from May 2014 to May 2015.

In [ ]:
df_housing.sort_values('date')

In [ ]:
# change "yr_built" dtype to datetime with format %Y
# df_housing['yr_built'] = pd.to_datetime(df_housing['yr_built'], format='%Y')

In [ ]:
# not relevant for Larry! Since more complicated because of missing values, refrain from changing the format.
# how to convert yr_renovated? wrong format 0, NaN
# change "yr_renovated" dtype to datetime with format %Y
#df_housing['yr_renovated'] = pd.to_datetime(df_housing['yr_renovated'], format='%Y')

In [ ]:
# Take a new look
df_housing.head()

The column sqft_basement contains string objects, we expect/want floats. We can try and convert the datatype in order to get information from the error message. It tells us that the column contains "?" as non-convertible entry. We take a look at the rows with "?" entry and can then change the entry to NaN. Afterwards, the data type of the column can be changed to float.

In [ ]:
# # sqft_basement contains string objects, we want floats.
# # try to convert it to get error message:
# # not relevant for Larry
# df_housing = df_housing.astype({'sqft_basement': float})
# # we get '?' as non-convertible entry

In [ ]:
#look at rows with '?' entry in sqft_basement column
df_housing.query('sqft_basement == "?"')

In [ ]:
#replace the `?`-character with a numpy NaN value
df_housing['sqft_basement'] = df_housing.sqft_basement.replace('?',np.NaN)
# change data type to float
df_housing = df_housing.astype({'sqft_basement': float})
df_housing.sqft_basement.dtypes

### Missing Values 

Let us take another look at the information summary, this time focusing on the missing values.

In [ ]:
df_housing.info()

In [ ]:
# looking at missing values
# display number of missing values per column
df_housing.isna().sum()
# Only waterfront is relevant for Larry's wishes. 
# It is reasonable to assume that for the houses with NaN, 
# it is not known whether they are located at the waterfront.
# the missing values cannot be imputed.
# the other columns can be deleted, no imputation necessary.

In [ ]:
print(f"numbers of rows : {df_housing.shape[0]}")
print(f"missing values in waterfront : {round(df_housing.waterfront.isna().sum()/df_housing.shape[0]*100,2)} %")
print(f"missing values in view : {round(df_housing.view.isna().sum()/df_housing.shape[0]*100,2)} %")
print(f"missing values in sqft_basement : {round(df_housing.sqft_basement.isna().sum()/df_housing.shape[0]*100,2)} %")
print(f"missing values in yr_renovated : {round(df_housing.yr_renovated.isna().sum()/df_housing.shape[0]*100,2)} %")

print(f"missing values in data frame : {round(df_housing.isna().sum().sum()/(df_housing.shape[0]*df_housing.shape[1])*100,2)} %")
# .sum() twice in last row: first gives a series of the number of nan-values per column, 
# the second sums these up

After we deleted a row earlier, we still have 21596 observations. Of these observations, 

* 11 % have missing values in waterfront
* 0.29 % have missing values in view
* 2.1 % have missing values in sqft_basement
* 17.79 % have missing values in yr_renovated

In total, only 1.49 % of the data in our DataFrame is missing.

It is reasonable to assume that the missing data was not available for the houses. 

The only characteristic with missing values relevant for our client is "waterfront". Since a house is either located at the waterfront or not and the location of other houses does not imply whether even very closely located houses are waterfront houses, we cannot impute the information. 

The other missing values are not of relevance to our further investigation.

Hence, we will keep the missing values as they are. 

In [ ]:
# plotting percentage of missing values per column
# msno.bar(df_housing)

In [ ]:
# msno.matrix(df_housing)

## Research Questions and Hypothesis Generation

### Questions with belonging hypotheses and their indicators:

* Does the location of a house affect the price?
    1. If a house is located close to water, then the price is higher (waterfront(yes/no)) 
    1. The more central a house is, the higher the price (location)

* Does the size of a house affect the price?
    1. The more bedrooms a house has, the higher the price (bedrooms)
    1. The higher the square footage of the house, the higher the price (sqft_liviing)

* Does the size of the lot affect the price?
    1. The larger the lot, the higher the price (sqft_lot)

* Does the state the house is in affect the price?
    1. The better the overall condition of the house, the higher the price (condition)
    1. The better the grade, the higher the price (grade)


### Questions relevant for Client

Client: Larry Sanders, Buyer.
Characteristics: Waterfront , limited budget, nice & isolated but central neighborhood without kids (but got some of his own, just doesn't want his kids to play with other kids .. because of germs)

How should we parametrize Larry's wishes? While some criteria, like the wish for waterfront housing, are self evident, others are not. The "nice house" criterion could be parametrized by "condition" and "grade". Before we decide on a limit, let us take a look at their distribution:

In [ ]:
sns.histplot(data=df_housing, 
                x='condition', 
                palette=["cadetblue"],
                ).set_title('Condition of the Houses')
plt.xlabel('Condition')
plt.show();

In [ ]:
sns.histplot(data=df_housing, 
                x='grade', 
                palette=["cadetblue"],
                ).set_title('Grade of the Houses')
plt.xlabel('Grade')
plt.show();

Based on the above computations of the median for "grade" and "condition" and our view of the distribution of these values, we decide on the following parametrization.

Our Assumptions:

- Waterfront - evident by yes/no attribute
- Limited budget - price not above median
- Nice house - condition 3 and up, grade 7 and up ( = median)
- Isolated house - sufficiently large lot size, sufficiently large lot size of the 15 nearest neighbors. Above median
- Central neighborhood - Top 15 zip codes with the highest population density
- Neighborhood without kids - zip codes with at most 2 schools
- Has children - at least 2 bedrooms

What springs to mind: Are there any houses that meet all the criteria?

* Room for kids: Does the size of a house affect the price?
    1. The more bedrooms a house has, the higher the price (bedrooms)

* Nice house: Does the state the house is in affect the price?
    1. The better the overall condition of the house, the higher the price (condition)
    1. The better the grade, the higher the price (grade)

* Waterfront, central: Does the location of a house affect the price?
    1. The more central a house is, the higher the price (location, zip code)
    1. If a house is located at the waterfront, then the price is higher (waterfront(yes/no)) 

* Does the size of the lot affect the price?
    1. The larger the lot, the higher the price. (sqft_lot)

### categorical data:

- id
- date
- bedrooms
- bathrooms
- sqft_living
- sqft_lot
- floors
- waterfront
- view
- condition
- grade
- yr_built
- yr_renovated
- zipcode

### continuous data:

- price
- sqft_above
- sqft_basement
- lat
- long
- sqft_living15
- sqft_lot15

### Hypothesis: Houses located at the waterfront have a higher price

Let us look at the characteristics "waterfront" and "price".

In [ ]:
counts_waterfront = df_housing.waterfront.value_counts(dropna=False)

#df_housing.groupby("waterfront", dropna=False).count()

#plt.hist(df_housing.waterfront.replace(np.nan, "unknown"))
#plt.show()

In [ ]:
df_waterfront = pd.DataFrame(counts_waterfront).reset_index()
df_waterfront.columns = ['waterfront', 'count']
df_waterfront.waterfront = ['no', 'unknown', 'yes']
df_waterfront

In [ ]:
# fig, ax1 = plt.subplots()#figsize =(10,6))
# sns.barplot(data=df_waterfront, x='waterfront', y='count', color='cadetblue')

# fig.suptitle("Distribution Waterfront", fontsize = 20)
# plt.subplots_adjust(top=0.90)
# ax1.set_xlabel("Waterfront", labelpad = 15, fontsize=12)
# ax1.set_ylabel("Count", labelpad = 15, fontsize=12)
# ax1.bar_label(ax1.containers[0]);

In [ ]:
fig = px.bar(df_waterfront, 
             x='waterfront', 
             y='count', 
             title="Waterfront?", 
             text='count',
             width=400, height=400)

# , color='cadetblue'
fig.update_layout(
     xaxis_title="", 
     yaxis_title="Count",
     font_family="Average",
     font_color="dimgray",
     font_size=20,
     title_font_family="Average",
     title_font_size=25,
     margin=dict(l=75, r=45, t=50, b=45),
#     paper_bgcolor="white",
     plot_bgcolor="white"
)
# texttemplate can be configured with rounding etc., e.g. '%{text:.2s}'
fig.update_traces(texttemplate='%{text}',marker=dict(color="cadetblue"),selector=dict(type="bar"))
fig.update_yaxes(showline=True, linecolor='dimgrey')
fig.update_xaxes(showline=True, linecolor='dimgrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

fig.show()

In [ ]:
#number of waterfront houses compared to total:
waterfront_houses = (df_housing.waterfront.values == 1).sum()
total_houses = df_housing.shape[0]
print(f"There are {waterfront_houses} waterfront houses and {total_houses} houses in total. Hence, only {waterfront_houses/total_houses * 100} % of the houses are waterfront houses. This wish highly restricts the available houses.")

In [ ]:
cor_price_water = df_housing.price.corr(df_housing.waterfront)
cor_price_water

The computed correlation between Waterfront location and price is unexpectedly low at 0.2763. However, there are only 146 waterfront houses and 21596 houses in total. Hence, only 0.6761 % of the houses are waterfront houses. The data on waterfront houses is very scarce.

We definitely learn that this wish highly restricts the available houses.

In [ ]:
# fig = px.scatter(df_housing, 
#              x='waterfront', 
#              y='price', 
# #             title="Waterfront - Price", 
# #             text='count',
#              width=400, height=400)

# # , color='cadetblue'
# fig.update_layout(
#      xaxis_title="Waterfront", 
#      yaxis_title="Price",
#      font_family="Average",
#      font_color="dimgray",
#      font_size=20,
#      title_font_family="Average",
#      title_font_size=25,
#      margin=dict(l=75, r=45, t=50, b=45),
# #     paper_bgcolor="white",
#      plot_bgcolor="white"
# )
# # texttemplate can be configured with rounding etc., e.g. '%{text:.2s}'
# fig.update_traces(marker=dict(color="cadetblue"))
# fig.update_yaxes(showline=True, linecolor='dimgrey')
# fig.update_xaxes(showline=True, linecolor='dimgrey')
# fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

# fig.show()

When we use additional markers, e.g. bedrooms or sqft_living, to compare similar houses located at/away from waterfront, we obtain new insights.

The mean, median for the price when waterfront yes/no separately confirm this:
* No waterfront:
    - Price mean 532636.36
    - Price median 450000.00
* Waterfront:
    - Price mean 1717214.73
    - Price median 1510000.00


### Hypothesis: The more bedrooms a house has, the higher the price (bedrooms)

In [ ]:
# Hypothesis: The more bedrooms a house has, the higher the price (bedrooms)
df_housing.waterfront = df_housing.waterfront.astype(str) # for discrete color scale
# change entries for nicer legend
df_housing['waterfront'] = df_housing['waterfront'].map({
    'nan': 'Unknown',
    '0.0': 'No', 
    '1.0': 'Yes'
    })
ax = sns.scatterplot(data=df_housing, 
                x='bedrooms', 
                y='price', 
                hue="waterfront",
                palette=["gray", "cadetblue", "darkslategrey"], 
#                legend=False,
                ).set_title('Bedrooms vs. Price')
#handles, labels  =  ax.get_legend_handles_labels()
#ax.legend(handles, ['unknown', 'No', 'Yes'])#, loc='lower right')
#plt.legend(title='Waterfront', loc='upper right', labels=['unknown', 'No', 'Yes'])
plt.legend(title='Waterfront')
plt.xlabel('Number of Bedrooms')
plt.ylabel('Price')
plt.show()
# change entries back
df_housing['waterfront'] = df_housing['waterfront'].map({
    'Unknown':'nan',
    'No':'0.0', 
    'Yes':'1.0'
#    np.NaN: 'fog'
    })
# change data type back to float
df_housing.waterfront = df_housing.waterfront.astype(float)

It seems from above plot that we might gather some insight from looking at waterfront houses only. The following plots support this assumption and underline the positive correlation of price and bedrooms, sqft_living for waterfront houses.

In [ ]:
# Hypothesis: The more bedrooms a house has, the higher the price (bedrooms)
ax = sns.lmplot(data=df_housing.query('waterfront==1'), 
                x='bedrooms', 
                y='price',
#                x_jitter=0.1, 
                scatter_kws={'color' : 'cadetblue'},
                line_kws={'color' : 'darkslategrey'}
                )#.set_title('Bedrooms vs. Price')
plt.xlabel('Number of Bedrooms')
plt.ylabel('Price')
plt.show()

sns.lmplot(data=df_housing.query('bedrooms<=6'), x='bedrooms', y='price',# hue="waterfront",
                scatter_kws={'color' : 'cadetblue'},
                line_kws={'color' : 'darkslategrey'}
                )#.set_title('sqft_living vs. Price')
plt.xlabel('Bedrooms')
plt.ylabel('Price')
#plt.title('sqft Living vs. Price')
plt.show();

In [ ]:
df_housing.price.corr(df_housing.bedrooms)

In [ ]:
df_housing.query('waterfront==1')['bedrooms'].corr(df_housing.query('waterfront==1')['price'])

In [ ]:
df_housing.query('bedrooms<=6')['bedrooms'].corr(df_housing.query('bedrooms<=6')['price'])

The correlation between price and number of bedrooms for waterfronthouses is strongly positive at 0.51.

In [ ]:
# change datatype to string for discrete color scale
df_housing.waterfront = df_housing.waterfront.astype(str) 
# change entries for nicer legend
df_housing['waterfront'] = df_housing['waterfront'].map({
    'nan': 'Unknown',
    '0.0': 'No', 
    '1.0': 'Yes'
    })
sns.scatterplot(data=df_housing, 
                x='sqft_living', 
                y='price', 
                hue="waterfront",
                palette=["gray", "cadetblue", "darkslategrey"], 
#                legend=False,
                ).set_title('sqft_living vs. Price')

plt.legend(title='Waterfront')#, labels=['','unknown', 'No', 'Yes'])

plt.xlabel('sqft Living')
plt.ylabel('Price')
#plt.title('sqft Living vs. Price')

sns.lmplot(data=df_housing, x='sqft_living', y='price', hue="waterfront",
                palette=["gray", "cadetblue", "darkslategrey"], 
#                legend=False,
                )#.set_title('sqft_living vs. Price')
plt.xlabel('sqft Living')
plt.ylabel('Price')
#plt.title('sqft Living vs. Price')
plt.show();
# change entries back
df_housing['waterfront'] = df_housing['waterfront'].map({
    'Unknown':'nan',
    'No':'0.0', 
    'Yes':'1.0'
#    np.NaN: 'fog'
    })
# change data type back to float
df_housing.waterfront = df_housing.waterfront.astype(float)

The price is naturally influenced by various factors. We need to keep this in mind when examining correlations: All other things being equal, the more reliable the correlation.
If we, e.g., restrict ourselves to houses with sqft_living $\in [4000,6000]$, then the correlation between price and waterfront is 0.39 - a strong increase compared to 0.28 before.

In [ ]:
#sqft_living in range 3000-5000, then look at correlation between price and waterfront
df_housing.query("sqft_living >= 4000 and sqft_living <= 6000").price.corr(df_housing.query("sqft_living >= 4000 and sqft_living <= 6000").waterfront)

In [ ]:
df_housing.groupby("waterfront")["price", "bedrooms"].describe()
#bedroom data comparable at/away from waterfront
#price is higher when at waterfront

In [ ]:
#pd.plotting.scatter_matrix(df_housing[["price", "waterfront", "condition", "grade", "bedrooms", "sqft_lot"]]);
sns.pairplot(df_housing[["price", "waterfront", "condition", "grade", "bedrooms", "sqft_lot", "sqft_lot15"]], dropna=False)

In [ ]:
#add column price per sqft_living
df_housing["price_per_sqft_living"] = df_housing.price / df_housing.sqft_living

How are the characteristics correlated?

high (> 0.5) positiv correlation between

- price and bathrooms
- price and grade
- price and sqft_living
- price and sqft_living15
- bedrooms and bathrooms
- bedrooms and sqft_living
- bathrooms and sqft_living
- bathrooms and floors
- bathrooms and grade
- bathrooms and sqft_above
- bathrooms and yr_built
- bathrooms and sqft_living15
- sqft_living and grade
- sqft_living and sqft_above
- sqft_living and sqft_living15
- sqft_lot and sqft_lot15
- floors and sqft_above
- grade and sqft_above
- grade and sqft_living15
- sqft_above and sqft_living15


In [ ]:
df_housing.corr().style.bar(align='zero',color=["orange"])

In [ ]:
# Hypothesis: The more bedrooms a house has, the higher the price (bedrooms)
df_housing.waterfront = df_housing.waterfront.astype(str) # for discrete color scale
ax = sns.scatterplot(data=df_housing, 
                x='bedrooms', 
                y='price_per_sqft_living', 
                hue="waterfront",
#                palette='crest', 
                palette=("Greys")
#                legend=False,
                ).set_title('Bedrooms vs. Price/sqft_living')
#handles, labels  =  ax.get_legend_handles_labels()
#ax.legend(handles, ['unknown', 'No', 'Yes'])#, loc='lower right')
#plt.legend(title='Waterfront', loc='upper right', labels=['unknown', 'No', 'Yes'])
plt.legend(title='Waterfront')
plt.xlabel('Number of Bedrooms')
plt.ylabel('Price/sqft_living')
plt.show()
df_housing.waterfront = df_housing.waterfront.astype(float)

Concerning the hypothesis "The more bedrooms, the higher the price", further investigation, especially of the larger houses, is needed.

The correlation of those two characteristics is 0.32. The distribution is unexpected for >= 6 bedrooms.

### Hypothesis: The better the overall condition /the higher the grade of the house, the higher the price


In [ ]:
# Hypothesis: The better the overall condition /the higher the grade of the house, the higher the price
sns.scatterplot(data=df_housing, x='condition', y='price', hue="grade", palette='Greys')
plt.legend(title='Grade')
plt.xlabel('Condition')
plt.ylabel('Price')
plt.show()

sns.scatterplot(data=df_housing, x='grade', y='price', hue="condition", palette='crest')
plt.legend(title='Condition')
plt.xlabel('Grade')
plt.ylabel('Price')
plt.show()
#sns.lmplot(data=df_housing, x='grade', y='price')#, hue="condition");

In [ ]:
# # Hypothesis: The better the overall condition /the higher the grade of the house, the higher the price
# sns.scatterplot(data=df_housing, x='condition', y='price_per_sqft_living', hue="grade")
# plt.show()
# sns.scatterplot(data=df_housing, x='grade', y='price_per_sqft_living', hue="condition")
# plt.show()
#sns.lmplot(data=df_housing, x='grade', y='price_per_sqft_living')#, hue="condition");

There are 788 observations where the sqft_living is larger than the sqft_lot. How is this possible? Imaginable are, e.g.,  multiple story units/houses on comparably small lots.

In [ ]:
df_housing.query("sqft_living > sqft_lot")#.value_counts()

In [ ]:
sns.scatterplot(data=df_housing, 
                x='sqft_lot', 
                y='price', 
                hue="sqft_lot15",
#                palette=["palevioletred", "cadetblue", "darkslategrey"], 
#                legend=False,
                ).set_title('sqft_lot vs. Price')

plt.legend(title='sqft_lot15')

plt.xlabel('sqft Lot')
plt.ylabel('Price')

## Answers to Hypotheses

1. the price is higher for houses at the waterfront. Yes.
1. the more bedrooms, the higher the price: true for waterfront housing. Only true up to and including 5 bedrooms in general
1. The better the overall condition /the higher the grade of the house, the higher the price. True for price/grade: correlation of 0.667951. not true for price/condition: correlation of 0.036056

## Assumptions for Larry:
Let us quickly recap our assumptions for our client Larry Sanders:

- Waterfront - evident by yes/no attribute
- Limited budget - not larger than median
- Nice house - condition 3 and up, grade 7 and up ( = median)
- Isolated house - sufficiently large lot size, sufficiently large lot size of the 15 nearest neighbors. Above median
- Central neighborhood - Top 15 zip codes with the highest population density
- Neighborhood without kids - zip codes with at most 2 schools
- Has children - at least 2 bedrooms

Based on our assumptions, we will drop the columns not needed for our investigation.

In [ ]:
#dropping columns not of relevance to Larry's needs
df_Larry = df_housing.drop(['bathrooms', 'floors', 'view', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated'], axis = 1)
df_Larry

## Importing Additional Data

In order to be able to make statements about centrality and "neighborhoods without children", we make use of additional data. Let us now imporrt those two data sources.

First, we import data on population density per zip code and transform the columns inzo a format we can work with.

In [ ]:
# get data on population density per zip code
# from http://www.usa.com/rank/king-county-wa--population-density--zip-code-rank.htm?yr=9000&dis=&wist=&plow=&phigh=
df_pop_density = pd.read_csv('data/King_County_Population_Density.csv', sep = '\t')

In [ ]:
df_pop_density

In [ ]:
# split zip/population column into two columns
df_pop_density[['zip','population']]= df_pop_density["zip/population"].str.split("/", expand=True)

In [ ]:
#drop combined column
df_pop_density.drop("zip/population", axis=1, inplace=True)

In [ ]:
#change data type of rank column to int
df_pop_density = df_pop_density.astype({'rank': int})

In [ ]:
# remove comma and unit /sq mi from population density column and turn into float
df_pop_density["population_density"] = df_pop_density.population_density.str.strip('/sq mi')

In [ ]:
#remove , from string numbers and turn into float
df_pop_density["population_density"] = df_pop_density.population_density.str.replace(',', "").astype('float')

In [ ]:
# zip into int
df_pop_density["zip"] = df_pop_density.zip.astype('int')

In [ ]:
#check our work
df_pop_density.head()

In [ ]:
df_pop_density.info()

In [ ]:
# drop columns not needed 
df_pop_density_short = df_pop_density.drop(["population_density","population"], axis=1)
df_pop_density_short

We now have a DataFrame consisting of King County zip codes and corresponding population density rank.

We store this information in a dictionary.

In [ ]:
ranks = df_pop_density_short['rank'].tolist()
dens_zips = df_pop_density['zip'].tolist()
dict_pop_rk = dict(zip(dens_zips,ranks)) 
dict_pop_rk 

We now import the data on schools in King County. We also transform this data so that we can work with it.

In [ ]:
# from https://gis-kingcounty.opendata.arcgis.com/datasets/kingcounty::school-sites-in-king-county-schsite-point/explore?location=47.503391%2C-122.188658%2C10.00
df_schools = pd.read_csv('data/School_Sites_in_King_County___schsite_point.csv')
df_schools.head()

In [ ]:
# drop not needed columns
df_schools.drop(['X','Y', 'FEATURE_ID', 'ESITE', 'CODE','ABB_NAME', 'OSPI_CODE', 'PIN', 'MAJOR', 'MINOR'], axis=1, inplace=True)

In [ ]:
# take a look
df_schools.head()

In [ ]:
# Check for duplicates - no duplicates!
df_schools.duplicated().value_counts()

In [ ]:
# count schools per zip code
df_schools.groupby('ZIPCODE').count()

We group by zip codes in order to obtain the number of schools per zip code. We reset the index to obtain the column 'zipcode' again. We then store this information in a dictionary.

In [ ]:
#store this information with reset index in order to still have a column 'zipcode'
df_schools_zip = df_schools.groupby('ZIPCODE').count().reset_index()[['ZIPCODE','OBJECTID']]
df_schools_zip

In [ ]:
# make dictionary of schools per zip code
dict_school_zip = dict(zip(df_schools_zip['ZIPCODE'], df_schools_zip['OBJECTID']))
dict_school_zip

In [ ]:
# # get schools per zip code
# %store -r dict_school_zip
# print(dict_school_zip)
# # get population density rank of zipcodes
# %store -r dict_pop_rk
# print(dict_pop_rk)
# %store -r df_pop_density_short

In [ ]:
# pd.Series(dict_school_zip.values()).value_counts().describe()

In [ ]:
np.unique(np.array(sorted(dict_school_zip.values())))


The different zip code areas contain between 1 and 20 schools, 15 does not occur.

In [ ]:
sorted(dict_school_zip.values())

### New characteristics for our Data

We add two columns to 'Larrys DataFrame'. Both result as the image of a map on the zip codes by applying the dictionaries of population density rank and schools per zip code.

The column 'population density rank' will be our measure for centrality of the houses. The column 'schools per zip code' will be our measure for children free neighborhoods.

In [ ]:
df_Larry['density_rank']= df_Larry.zipcode.map(dict_pop_rk)
df_Larry['schools_per_zip'] = df_Larry.zipcode.map(dict_school_zip)
df_Larry

We can now examine the centrality of the houses listed.

In [ ]:
sns.scatterplot(data=df_Larry, x='density_rank', y='price', hue='schools_per_zip');

In [ ]:
sns.scatterplot(data=df_Larry, x='density_rank', y='price_per_sqft_living');#, hue='schools_per_zip')

In [ ]:
df_Larry[['price','density_rank','schools_per_zip']].corr().style.bar(align='zero',color=["orange"])

Contrary to our assumption, the centrality (based on population density) is not correlated to the price.

In [ ]:
sns.scatterplot(data=df_Larry, x='schools_per_zip', y='price', hue='density_rank');

In [ ]:
sns.scatterplot(data=df_Larry, x='density_rank', y='schools_per_zip', hue='price');

In [ ]:
df_Larry.corr()

## A Wish Function for Larry

What are we to recommend to our client? A function to the rescue! Larry has 7 wishes, hence, there are 7 criteria. Each criterion satisfied counts for 1. 
If 2 variables determine a single criterion (e.g., grade and condition determine the state "nice" of the house), each accounts for 0.5.

By definition, the maximum possible outcome per house is 7 points.

__Goal: recommend zip code for Larry__ where he has most chances of finding a house he likes.

points for:

* waterfront yes
* limited budget: price not larger than median (not mean since too highly influenced by extreme values), i.e. 450000 $
* nice: 
    - Condition: An index from 1 to 5 on the condition of the apartment (overall). Seems to be no correlation, but for Larry we can decide on only looking at Condition 3 and up since he wants a nice house.
    - Grade: An index from 1 to 13, where 1-3 falls short of building construction and design, 7 has an average level of construction and design, and 11-13 have a high quality level of construction and design. Highly correlated to price. For Larry, we will only consider grade 7 and up housing since he wants a nice house.
    - to weigh every wish equally, both measure 0.5
* isolated: 
    - sqft_lot above median so that house has above average distance to neighbor, i.e. 7619 sqft
    - sqft_lot15 above median so that 15 nearest neighbors also have above average distance, i.e. 7620 sqft
    - to weigh every wish equally, both measure 0.5
* central: when in top 15 (or max_rank) population density zip codes, i.e. if density_rank <= 15
* room for kids: bedrooms >= 2
* neighborhood without kids: no more than 2 (1st quartile) schools in the zip code

In [ ]:
# collect numbers needed for function
price_limit = df_housing.price.median()
price_limit

In [ ]:
lot_min = df_housing.sqft_lot.median()
lot_min

In [ ]:
lot15_min = df_housing.sqft_lot15.median()
lot15_min

In [ ]:
# #list of top 15 population density rank zip codes
# # truncate DataFrame after top 15
# df_pop_density_short[:15]
# # add zip entries to list
# ls_central = df_pop_density_short[:15].zip.tolist()
# ls_central

We are now ready to define the wish function for Larry. Based on this function, we make a new column in our DataFrame that contains the number of attained points for every observation.

In [ ]:
#define the function that assigns the number of fulfilled wishes to the listed houses - the 'wish points'
def larrys_wishes(price, condition, grade, sqft_lot, sqft_lot15, density_rank, bedrooms, schools_per_zip, waterfront=0, price_limit=450000, condition_min=3, grade_min=7, lot_min=7619, lot15_min=7620.0, max_rank=15, bedrooms_min=2, max_schools=2):
    """computes the 'wish points' of house observations according to Larrys wishes, where each of his seven wishes accounts for 1 point. value between 0 and 7.

    Args:
        price (float): price of the house sold
        condition (int): condition  of the house sold
        grade (int): grade of the house sold
        sqft_lot (float): size of the lot of the house sold
        sqft_lot15 (float): average lot size of the 15 nearest neighbors of the house sold
        density_rank (int): population density rank of the zip code the house is located in, 1 being the highest
        bedrooms (int): number of bedrooms of the house sold
        schools_per_zip (int): number of schools located in the zip code of the house sold
        waterfront (int, optional): 1 for waterfront location, 0 for no waterfront location. Defaults to 0.
        price_limit (int, optional): Larry's price limit in order to meet his criteria. Defaults to 450000.
        condition_min (int, optional): Condition contributes a half wish point if not lower than condition_min. Defaults to 3.
        grade_min (int, optional): Grade contributes a half wish point if not lower than grade_min. Defaults to 7.
        lot_min (int, optional): sqft_lot contributes a half wish point if not lower than lot_min. Defaults to 7619.
        lot15_min (float, optional): sqft_lot15 contributes a half wish point if not lower than lot15_min. Defaults to 7620.0.
        max_rank (int, optional): density_rank contributes a wish point if not above max_rank. Defaults to 15.
        bedrooms_min (int, optional): bedrooms contributes a wish point if not lower than bedrooms_min. Defaults to 2.
        max_schools (int, optional): schools_per_zip contributes a wish point if not above max_schools. Defaults to 2.

    Returns:
        float: The number of Larry's wishes the observation fulfills. Half points stem from wishes determined by two criteria.
    """
    points = 0
    if waterfront == 1:
        points += 1
    if price <= price_limit:
        points += 1
    if condition >= condition_min:
        points += 0.5
    if grade >=grade_min:
        points += 0.5
    if sqft_lot > lot_min:
        points += 0.5
    if sqft_lot15 > lot15_min:
        points += 0.5
    if density_rank <= max_rank:
        points += 1
    if bedrooms >= bedrooms_min:
        points += 1
    if schools_per_zip <= max_schools:
        points += 1
    return points

In [ ]:
# make new column
df_Larry["wish_points"] = df_Larry.apply(lambda row: larrys_wishes(row.price, row.condition, row.grade, row.sqft_lot, row.sqft_lot15, row.density_rank, row.bedrooms, row.schools_per_zip, row.waterfront), axis=1)

In [ ]:
#see if it worked
df_Larry.head()

In [ ]:
df_Larry.describe()['wish_points']

In [ ]:
(
df_Larry.groupby("zipcode")
    .mean()
    .sort_values(['wish_points'], ascending=False)
)

We see that the maximum value of points attained by houses sold is 5. The maximum average wish points per zip code are 4.4.

However, a house fulfilling all of Larry's wishes should have 7 points. 

__Not a single house meeting all of Larry's criteria has been sold.__

Thus, we decide to have an imaginary talk with our clint in order to discuss which criteria to relax.

Since the price restriction is non negotiable (there just is no more money...) this restriction is kept as is.

We decide to omit the requirements for "waterfront" since this is simply too big a restriction.

We relax the requirements for "neighborhood without kids" to 4 schools per zip code and to relax the requirements for the lot size of the house and those of of the neighbors since he would still have a sufficiently large lot himself (and does not have to let anyone in, anyway). Values changed to 30th percentile: sqft_lot 5612.000, sqft_lot15 5625.500

We furthermore decide to look somewhat less central since Larry is not too fond of other people anyways. (maximal population density rank raised to 25)

Grade and condition are lowered to 6 and 2, resp.

The new maximum value for his wish function is, hence, 6.

In [ ]:
#get value of 30th percentile of lot size
df_Larry[['sqft_lot', 'sqft_lot15']].quantile(.3)

In [ ]:
# we use the function defined before with changed limit values to obtain a new column with "relaxed wish points"
# omit waterfront so that default value 0 is used.
df_Larry["wish_points_rel"] = df_Larry.apply(lambda row: larrys_wishes(
    row.price, 
    row.condition, 
    row.grade, 
    row.sqft_lot, 
    row.sqft_lot15, 
    row.density_rank, 
    row.bedrooms, 
    row.schools_per_zip, 
    price_limit=450000, 
    condition_min=3, 
    grade_min=7, 
    lot_min=5612, 
    lot15_min=5625, 
    max_rank=25, 
    bedrooms_min=2, 
    max_schools=4), 
    axis=1
    )

In [ ]:
df_Larry.describe()['wish_points_rel']

In [ ]:
df_Larry.query("wish_points_rel == 6").groupby("zipcode").count()
#all 17 houses with the maximal possible number of 6 wish points lie in zip code 98136

In [ ]:
houses_98136 = df_Larry.zipcode.value_counts()[98136]
houses_98136_6 = df_Larry.query("wish_points_rel == 6").zipcode.value_counts()[98136]
print(f"There are {houses_98136_6} houses in zip code 98136 with maximal number of 6 wish points. A total of {houses_98136} houses have been sold in that zip code. Hence, {houses_98136_6 / houses_98136 * 100} % of the sold houses in that zip code fulfill all requirements.")

We find out that all 17 houses with the maximal possible number of 6 wish points lie in zip code __98136__. Hence, if Larry has his mind set on finding a house that meets all of the remaining criteria, he should definitely look there.

In [ ]:
df_Larry.wish_points_rel.hist(color=["cadetblue"])
plt.xlabel('Wish Points')
plt.ylabel('Count')

In case Larry might be willing to lessen his restrictions slightly, let us look at the average points of houses put on the market. How likely is the average house to fulfill Larry's wishes?

We look at the average points per zip code. 
The top ten thereof are our recommendation as to where to look for houses.

Note that we do not consider the value count `df_Larry.query("wish_points_rel>=5").value_counts("zipcode")` because we cannot base our recommendation on this count: They are absolute values, strongly influenced by the total number of houses in those zip codes.

In [ ]:
# How likely is the average house put on the market to fulfill Larry's wishes?
# we look at the average points per zip code. 
# The top ten thereof are our recommendation as to where to look for houses.
df_Larry_rec = pd.DataFrame(
df_Larry.groupby("zipcode")
    .mean()
    .sort_values(['wish_points_rel'], ascending=False)['wish_points_rel']#[:11]
)
df_Larry_rec.reset_index(inplace=True)
df_Larry_rec.head(10)

In [ ]:
df_Larry_rec.tail()

In [ ]:
# import plotly.graph_objects as go

# #df = pd.read_csv("data_group_work/airports.csv")

# #namelist = [f'IATA: {df["iata"][x]}<br>Name: {df["name"][x]}<br>State: {df["state"][x]}' for x in range(len(df))] # We need this later for the 'text'-argument in go.Scattergeo() to make the labels look nicer.

# fig = go.Figure(

# go.Scattergeo(
#         locationmode = 'USA-states', 
#         lon = df_housing['long'],
#         lat = df_housing['lat'],
# #        text = namelist,
#         mode = 'markers',
#         marker = dict( # controls the points
#             size = 2,
#             color = 'red',
#             opacity = 1
#         )
#     ))

#fig.add_trace() add county/zip bordrs as trace

# fig.update_layout(
#          title_text = 'Houses sold',
#          showlegend = False,
#          margin={"r":0,"t":50,"l":0,"b":0},
#          geo = dict(
#              scope = 'usa',
#              landcolor = 'rgb(217, 217, 217)'
#          )
#      )

# fig.update_geos(fitbounds='locations')

# fig.show()

## Recommendations for Larry

1. Don’t restrict yourself to waterfront housing!
1. If all other criteria should be satisfied: Zip code 98136! All 26 Houses with maximal possible 6 wish points are there
1. Top 10 zip codes with highest average “wish fulfilling property”

|Zip codes |Average wish points|
|---|---|
| 98030 | 4.86|
| 98031 | 4.84 |
| 98136 | 4.83 |
| 98045 | 4.54 |
| 98010 | 4.50 |
| 98055 | 4.47 |
| 98133 | 4.46 |
| 98146 | 4.39 |
| 98125 | 4.36 |
| 98024 | 4.35 |


In [ ]:
# take a look at the number of houses sold per zip code
df_housing.groupby('zipcode').count().reset_index().sort_values('id', ascending=False)

In [ ]:
# additional requirements for map plots
import plotly.express as px
from urllib.request import urlopen
import json

In [ ]:
with urlopen('https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/wa_washington_zip_codes_geo.min.json') as response:
#with urlopen('http://data-seattlecitygis.opendata.arcgis.com/datasets/83fc2e72903343aabff6de8cb445b81c_2.geojson') as response:
# data from https://catalog.data.gov/dataset/zip-codes-2259a
# data/Zip_Codes.geojson
#with urlopen('https://data-seattlecitygis.opendata.arcgis.com/datasets/SeattleCityGIS::zip-codes.geojson?outSR=%7B%22latestWkid%22%3A2926%2C%22wkid%22%3A2926%7D') as response:
    zipcodes = json.load(response)
fig = px.choropleth(df_housing.groupby('zipcode').count().reset_index(),
                    geojson=zipcodes, 
                    locations='zipcode', 
                    color='id',
                    color_continuous_scale="bluyl",#"Viridis_r",
#                    range_color=(3.5,5),
                    featureidkey="properties.ZCTA5CE10",
                    scope="usa",
                    labels={'id':'Number of Houses Sold'}
                          )

fig.update_layout(
    title = dict(text='Houses Sold', y=0.9, yanchor='top'),
    margin={"r":50,"t":50,"l":50,"b":50}
    )
fig.update_geos(fitbounds='locations')
fig.show()

We want to hand Larry a map of the recommended zip codes so that he knows right away where to start looking.

We import geo data of the zip code area boundaries in order to plot them. the areas will be colored according to the average wish point value.

In [ ]:
with urlopen('https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/wa_washington_zip_codes_geo.min.json') as response:
    zipcodes = json.load(response)
fig = px.choropleth(df_Larry_rec,
                    geojson=zipcodes, 
                    locations='zipcode', 
                    color='wish_points_rel',
                    color_continuous_scale="bluyl",#"Viridis_r",
                    range_color=(2.5,5),#set boundaries for colors
                    featureidkey="properties.ZCTA5CE10",
                    scope="usa",
                    labels={'wish_points_rel':'Average Wish Points'}
                          )

fig.update_layout(
    title = dict(text='Recommended Zip Codes for Larry', y=0.95, yanchor='top'),
    margin={"r":50,"t":50,"l":50,"b":50}
    )
fig.update_geos(fitbounds='locations')
fig.show()

In [ ]:
#Why is this not working????
#with urlopen('http://data-seattlecitygis.opendata.arcgis.com/datasets/83fc2e72903343aabff6de8cb445b81c_2.geojson') as response:
# data taken from https://catalog.data.gov/dataset/zip-codes-2259a
# data/Zip_Codes.geojson
#with urlopen('https://data-seattlecitygis.opendata.arcgis.com/datasets/SeattleCityGIS::zip-codes.geojson?outSR=%7B%22latestWkid%22%3A2926%2C%22wkid%22%3A2926%7D') as response:

# change to string in order to match geojson file
#df_Larry_rec['zipcode'] = df_Larry_rec['zipcode'].astype('str')
with open('data/Zipcodes_for_King_County_and_Surrounding_Area_(Shorelines)___zipcode_shore_area.geojson') as response:
# with open('data/Zip_Codes.geojson') as response:
    zipcodes = json.load(response)
fig = px.choropleth_mapbox(df_Larry_rec,
                    geojson=zipcodes, 
                    locations='zipcode', 
                    color='wish_points_rel',
                    color_continuous_scale="bluyl",#"Viridis_r",     
                    featureidkey="properties.ZIP",    
# featureidkey argument defines the property in geojson that is used to match to the locations argument value
#                    scope="usa",
                    labels={'wish_points_rel':'Average Wish Points'}
                          )

fig.update_layout(
    title = dict(text='Recommended Zip Codes for Larry', y=0.9, yanchor='top'),
    margin={"r":50,"t":50,"l":50,"b":50}
    )
fig.update_geos(fitbounds='locations')
fig.show()
#df_Larry_rec['zipcode'] = df_Larry_rec['zipcode'].astype('int')

In [ ]:
#Why is this not working????
# change to string in order to match geojson file
df_Larry_rec['zipcode'] = df_Larry_rec['zipcode'].astype('str')
print(len(df_Larry_rec['zipcode']))
with open('data/Zip_Codes.geojson') as response:
    
    zipcodes = json.load(response)
    fig = px.choropleth(df_Larry_rec,
                    geojson=zipcodes, 
                    locations='zipcode', 
                    color='wish_points_rel',
                    color_continuous_scale="bluyl",#"Viridis_r",     
                    featureidkey="properties.ZIPCODE",    
                    # featureidkey argument defines the property in geojson that is used to match to the locations argument value
                    scope="usa",
                    #labels={'wish_points_rel':'Average Wish Points'}
                          )

    fig.update_layout(
        title = dict(text='Recommended Zip Codes for Larry', y=0.9, yanchor='top'),
        margin={"r":50,"t":50,"l":50,"b":50}
        )
    fig.update_geos(fitbounds='locations')
    fig.show()
    df_Larry_rec['zipcode'] = df_Larry_rec['zipcode'].astype('int')

In [ ]:
len(zipcodes['features'])

## Future Work

The analysis can be further improved by considering the following aspects:

* Add weight options or ranks to the criteria in the wish function
* Nicer recommendation plot
* Build dashboard where you can change the weights/ranks
* Neighborhoods without kids:
    * Match school data to time frame sold: School data is from 2021
    * Find data on area per zip code,  compute “school density” for more precise ranking
    * Find/understand census data on age groups per zip code
* Alternative strategy concerning “central” houses
* Add renovation status as another factor for “nice” houses